## Redes Neuroanles  con regresión 

In [36]:
import numpy as np
import pandas as pd
import keras
from sklearn.model_selection import GridSearchCV
import time
import keras
from keras.models import Sequential
from keras.layers import Dense
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold


### Importando los datos

In [37]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [38]:
data_path = '/content/drive/MyDrive/Statistical/Laboratorio2/supermarket_sales.csv'
dataset = pd.read_csv(data_path)
dataset.head()

,Invoice ID,Branch,City,Customer type,Gender,Product line,Unit price,Quantity,Tax 5%,Total,Date,Time,Payment,cogs,gross margin percentage,gross income,Rating
0,750-67-8428,A,Yangon,Member,Female,Health and beauty,74.69,7,26.1415,548.9715,1/5/2019,13:08,Ewallet,522.83,4.761905,26.1415,9.1
1,226-31-3081,C,Naypyitaw,Normal,Female,Electronic accessories,15.28,5,3.8200,80.2200,3/8/2019,10:29,Cash,76.40,4.761905,3.8200,9.6
2,631-41-3108,A,Yangon,Normal,Male,Home and lifestyle,46.33,7,16.2155,340.5255,3/3/2019,13:23,Credit card,324.31,4.761905,16.2155,7.4
3,123-19-1176,A,Yangon,Member,Male,Health and beauty,58.22,8,23.2880,489.0480,1/27/2019,20:33,Ewallet,465.76,4.761905,23.2880,8.4
4,373-73-7910,A,Yangon,Normal,Male,Sports and travel,86.31,7,30.2085,634.3785,2/8/2019,10:37,Ewallet,604.17,4.761905,30.2085,5.3


In [310]:
dataset.shape

(1000, 17)

### Procesamiento de datos / Análisis de la data


In [39]:
dataset.head()

,Invoice ID,Branch,City,Customer type,Gender,Product line,Unit price,Quantity,Tax 5%,Total,Date,Time,Payment,cogs,gross margin percentage,gross income,Rating
0,750-67-8428,A,Yangon,Member,Female,Health and beauty,74.69,7,26.1415,548.9715,1/5/2019,13:08,Ewallet,522.83,4.761905,26.1415,9.1
1,226-31-3081,C,Naypyitaw,Normal,Female,Electronic accessories,15.28,5,3.8200,80.2200,3/8/2019,10:29,Cash,76.40,4.761905,3.8200,9.6
2,631-41-3108,A,Yangon,Normal,Male,Home and lifestyle,46.33,7,16.2155,340.5255,3/3/2019,13:23,Credit card,324.31,4.761905,16.2155,7.4
3,123-19-1176,A,Yangon,Member,Male,Health and beauty,58.22,8,23.2880,489.0480,1/27/2019,20:33,Ewallet,465.76,4.761905,23.2880,8.4
4,373-73-7910,A,Yangon,Normal,Male,Sports and travel,86.31,7,30.2085,634.3785,2/8/2019,10:37,Ewallet,604.17,4.761905,30.2085,5.3


In [40]:
# Procesamiento de datos
# Conocer los nombres de las variables
dataset.columns

Index(['Invoice ID', 'Branch', 'City', 'Customer type', 'Gender',
       'Product line', 'Unit price', 'Quantity', 'Tax 5%', 'Total', 'Date',
       'Time', 'Payment', 'cogs', 'gross margin percentage', 'gross income',
       'Rating'],
      dtype='object')

In [41]:
#verificación de NAs
dataset.isnull().values.any()

False

In [42]:
# Transfromaciones del pipeline de ingenieria de Características 
dataset.describe

<bound method NDFrame.describe of       Invoice ID Branch       City Customer type  Gender  \
0    750-67-8428      A     Yangon        Member  Female   
1    226-31-3081      C  Naypyitaw        Normal  Female   
2    631-41-3108      A     Yangon        Normal    Male   
3    123-19-1176      A     Yangon        Member    Male   
4    373-73-7910      A     Yangon        Normal    Male   
..           ...    ...        ...           ...     ...   
995  233-67-5758      C  Naypyitaw        Normal    Male   
996  303-96-2227      B   Mandalay        Normal  Female   
997  727-02-1313      A     Yangon        Member    Male   
998  347-56-2442      A     Yangon        Normal    Male   
999  849-09-3807      A     Yangon        Member  Female   

               Product line  Unit price  Quantity   Tax 5%      Total  \
0         Health and beauty       74.69         7  26.1415   548.9715   
1    Electronic accessories       15.28         5   3.8200    80.2200   
2        Home and lifestyl

In [43]:
dataset = dataset.iloc[:, 1:17]

In [44]:
dataset

,Branch,City,Customer type,Gender,Product line,Unit price,Quantity,Tax 5%,Total,Date,Time,Payment,cogs,gross margin percentage,gross income,Rating
0,A,Yangon,Member,Female,Health and beauty,74.69,7,26.1415,548.9715,1/5/2019,13:08,Ewallet,522.83,4.761905,26.1415,9.1
1,C,Naypyitaw,Normal,Female,Electronic accessories,15.28,5,3.8200,80.2200,3/8/2019,10:29,Cash,76.40,4.761905,3.8200,9.6
2,A,Yangon,Normal,Male,Home and lifestyle,46.33,7,16.2155,340.5255,3/3/2019,13:23,Credit card,324.31,4.761905,16.2155,7.4
3,A,Yangon,Member,Male,Health and beauty,58.22,8,23.2880,489.0480,1/27/2019,20:33,Ewallet,465.76,4.761905,23.2880,8.4
4,A,Yangon,Normal,Male,Sports and travel,86.31,7,30.2085,634.3785,2/8/2019,10:37,Ewallet,604.17,4.761905,30.2085,5.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,C,Naypyitaw,Normal,Male,Health and beauty,40.35,1,2.0175,42.3675,1/29/2019,13:46,Ewallet,40.35,4.761905,2.0175,6.2
996,B,Mandalay,Normal,Female,Home and lifestyle,97.38,10,48.6900,1022.4900,3/2/2019,17:16,Ewallet,973.80,4.761905,48.6900,4.4
997,A,Yangon,Member,Male,Food and beverages,31.84,1,1.5920,33.4320,2/9/2019,13:22,Cash,31.84,4.761905,1.5920,7.7
998,A,Yangon,Normal,Male,Home and lifestyle,65.82,1,3.2910,69.1110,2/22/2019,15:33,Cash,65.82,4.761905,3.2910,4.1


In [45]:
dataset['Branch'].value_counts()

A    340
B    332
C    328
Name: Branch, dtype: int64

In [46]:
dataset['City'].value_counts()

Yangon       340
Mandalay     332
Naypyitaw    328
Name: City, dtype: int64

In [47]:
dataset['Gender'].value_counts()

Female    501
Male      499
Name: Gender, dtype: int64

In [48]:
dataset['Product line'].value_counts()

Fashion accessories       178
Food and beverages        174
Electronic accessories    170
Sports and travel         166
Home and lifestyle        160
Health and beauty         152
Name: Product line, dtype: int64

In [49]:
dataset['Customer type'].value_counts()

Member    501
Normal    499
Name: Customer type, dtype: int64

In [50]:
dataset['Payment'].value_counts()

Ewallet        345
Cash           344
Credit card    311
Name: Payment, dtype: int64

In [51]:
#One Hot Encoding
ohe = pd.get_dummies(dataset['Branch'], drop_first=True)
dataset.drop('Branch', axis=1, inplace=True)
dataset = dataset.join(ohe)
dataset.head()

,City,Customer type,Gender,Product line,Unit price,Quantity,Tax 5%,Total,Date,Time,Payment,cogs,gross margin percentage,gross income,Rating,B,C
0,Yangon,Member,Female,Health and beauty,74.69,7,26.1415,548.9715,1/5/2019,13:08,Ewallet,522.83,4.761905,26.1415,9.1,0,0
1,Naypyitaw,Normal,Female,Electronic accessories,15.28,5,3.8200,80.2200,3/8/2019,10:29,Cash,76.40,4.761905,3.8200,9.6,0,1
2,Yangon,Normal,Male,Home and lifestyle,46.33,7,16.2155,340.5255,3/3/2019,13:23,Credit card,324.31,4.761905,16.2155,7.4,0,0
3,Yangon,Member,Male,Health and beauty,58.22,8,23.2880,489.0480,1/27/2019,20:33,Ewallet,465.76,4.761905,23.2880,8.4,0,0
4,Yangon,Normal,Male,Sports and travel,86.31,7,30.2085,634.3785,2/8/2019,10:37,Ewallet,604.17,4.761905,30.2085,5.3,0,0


In [52]:
#One Hot Encoding 
ohe = pd.get_dummies(dataset['Gender'], drop_first=True)
dataset.drop('Gender', axis=1, inplace=True)
dataset = dataset.join(ohe)
dataset.head()

,City,Customer type,Product line,Unit price,Quantity,Tax 5%,Total,Date,Time,Payment,cogs,gross margin percentage,gross income,Rating,B,C,Male
0,Yangon,Member,Health and beauty,74.69,7,26.1415,548.9715,1/5/2019,13:08,Ewallet,522.83,4.761905,26.1415,9.1,0,0,0
1,Naypyitaw,Normal,Electronic accessories,15.28,5,3.8200,80.2200,3/8/2019,10:29,Cash,76.40,4.761905,3.8200,9.6,0,1,0
2,Yangon,Normal,Home and lifestyle,46.33,7,16.2155,340.5255,3/3/2019,13:23,Credit card,324.31,4.761905,16.2155,7.4,0,0,1
3,Yangon,Member,Health and beauty,58.22,8,23.2880,489.0480,1/27/2019,20:33,Ewallet,465.76,4.761905,23.2880,8.4,0,0,1
4,Yangon,Normal,Sports and travel,86.31,7,30.2085,634.3785,2/8/2019,10:37,Ewallet,604.17,4.761905,30.2085,5.3,0,0,1


In [53]:
#One Hot Encoding 
ohe = pd.get_dummies(dataset['Payment'], drop_first=True)
dataset.drop('Payment', axis=1, inplace=True)
dataset = dataset.join(ohe)
dataset.head()

,City,Customer type,Product line,Unit price,Quantity,Tax 5%,Total,Date,Time,cogs,gross margin percentage,gross income,Rating,B,C,Male,Credit card,Ewallet
0,Yangon,Member,Health and beauty,74.69,7,26.1415,548.9715,1/5/2019,13:08,522.83,4.761905,26.1415,9.1,0,0,0,0,1
1,Naypyitaw,Normal,Electronic accessories,15.28,5,3.8200,80.2200,3/8/2019,10:29,76.40,4.761905,3.8200,9.6,0,1,0,0,0
2,Yangon,Normal,Home and lifestyle,46.33,7,16.2155,340.5255,3/3/2019,13:23,324.31,4.761905,16.2155,7.4,0,0,1,1,0
3,Yangon,Member,Health and beauty,58.22,8,23.2880,489.0480,1/27/2019,20:33,465.76,4.761905,23.2880,8.4,0,0,1,0,1
4,Yangon,Normal,Sports and travel,86.31,7,30.2085,634.3785,2/8/2019,10:37,604.17,4.761905,30.2085,5.3,0,0,1,0,1


In [326]:
# Cambio de datos a Númerico
#One Hot Encoding 
#datos['Gender'].replace(['female','male' ], [0,1], inplace = True)
#datos['Branch'].replace(['A','B', 'C' ], [0,1,2], inplace = True)


In [54]:
#One Hot Encoding 
ohe = pd.get_dummies(dataset['Customer type'], drop_first=True)
dataset.drop('Customer type', axis=1, inplace=True)
dataset = dataset.join(ohe)
dataset.head()

,City,Product line,Unit price,Quantity,Tax 5%,Total,Date,Time,cogs,gross margin percentage,gross income,Rating,B,C,Male,Credit card,Ewallet,Normal
0,Yangon,Health and beauty,74.69,7,26.1415,548.9715,1/5/2019,13:08,522.83,4.761905,26.1415,9.1,0,0,0,0,1,0
1,Naypyitaw,Electronic accessories,15.28,5,3.8200,80.2200,3/8/2019,10:29,76.40,4.761905,3.8200,9.6,0,1,0,0,0,1
2,Yangon,Home and lifestyle,46.33,7,16.2155,340.5255,3/3/2019,13:23,324.31,4.761905,16.2155,7.4,0,0,1,1,0,1
3,Yangon,Health and beauty,58.22,8,23.2880,489.0480,1/27/2019,20:33,465.76,4.761905,23.2880,8.4,0,0,1,0,1,0
4,Yangon,Sports and travel,86.31,7,30.2085,634.3785,2/8/2019,10:37,604.17,4.761905,30.2085,5.3,0,0,1,0,1,1


In [55]:
#One Hot Encoding 
ohe = pd.get_dummies(dataset['Product line'], drop_first=True)
dataset.drop('Product line', axis=1, inplace=True)
dataset = dataset.join(ohe)
dataset.head()

,City,Unit price,Quantity,Tax 5%,Total,Date,Time,cogs,gross margin percentage,gross income,...,C,Male,Credit card,Ewallet,Normal,Fashion accessories,Food and beverages,Health and beauty,Home and lifestyle,Sports and travel
0,Yangon,74.69,7,26.1415,548.9715,1/5/2019,13:08,522.83,4.761905,26.1415,...,0,0,0,1,0,0,0,1,0,0
1,Naypyitaw,15.28,5,3.8200,80.2200,3/8/2019,10:29,76.40,4.761905,3.8200,...,1,0,0,0,1,0,0,0,0,0
2,Yangon,46.33,7,16.2155,340.5255,3/3/2019,13:23,324.31,4.761905,16.2155,...,0,1,1,0,1,0,0,0,1,0
3,Yangon,58.22,8,23.2880,489.0480,1/27/2019,20:33,465.76,4.761905,23.2880,...,0,1,0,1,0,0,0,1,0,0
4,Yangon,86.31,7,30.2085,634.3785,2/8/2019,10:37,604.17,4.761905,30.2085,...,0,1,0,1,1,0,0,0,0,1


In [56]:
#One Hot Encoding 
ohe = pd.get_dummies(dataset['City'], drop_first=True)
dataset.drop('City', axis=1, inplace=True)
dataset = dataset.join(ohe)
dataset.head()

,Unit price,Quantity,Tax 5%,Total,Date,Time,cogs,gross margin percentage,gross income,Rating,...,Credit card,Ewallet,Normal,Fashion accessories,Food and beverages,Health and beauty,Home and lifestyle,Sports and travel,Naypyitaw,Yangon
0,74.69,7,26.1415,548.9715,1/5/2019,13:08,522.83,4.761905,26.1415,9.1,...,0,1,0,0,0,1,0,0,0,1
1,15.28,5,3.8200,80.2200,3/8/2019,10:29,76.40,4.761905,3.8200,9.6,...,0,0,1,0,0,0,0,0,1,0
2,46.33,7,16.2155,340.5255,3/3/2019,13:23,324.31,4.761905,16.2155,7.4,...,1,0,1,0,0,0,1,0,0,1
3,58.22,8,23.2880,489.0480,1/27/2019,20:33,465.76,4.761905,23.2880,8.4,...,0,1,0,0,0,1,0,0,0,1
4,86.31,7,30.2085,634.3785,2/8/2019,10:37,604.17,4.761905,30.2085,5.3,...,0,1,1,0,0,0,0,1,0,1


In [57]:
dataset= dataset.drop(['Date','Time'], axis=1)


In [58]:
dataset

,Unit price,Quantity,Tax 5%,Total,cogs,gross margin percentage,gross income,Rating,B,C,...,Credit card,Ewallet,Normal,Fashion accessories,Food and beverages,Health and beauty,Home and lifestyle,Sports and travel,Naypyitaw,Yangon
0,74.69,7,26.1415,548.9715,522.83,4.761905,26.1415,9.1,0,0,...,0,1,0,0,0,1,0,0,0,1
1,15.28,5,3.8200,80.2200,76.40,4.761905,3.8200,9.6,0,1,...,0,0,1,0,0,0,0,0,1,0
2,46.33,7,16.2155,340.5255,324.31,4.761905,16.2155,7.4,0,0,...,1,0,1,0,0,0,1,0,0,1
3,58.22,8,23.2880,489.0480,465.76,4.761905,23.2880,8.4,0,0,...,0,1,0,0,0,1,0,0,0,1
4,86.31,7,30.2085,634.3785,604.17,4.761905,30.2085,5.3,0,0,...,0,1,1,0,0,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,40.35,1,2.0175,42.3675,40.35,4.761905,2.0175,6.2,0,1,...,0,1,1,0,0,1,0,0,1,0
996,97.38,10,48.6900,1022.4900,973.80,4.761905,48.6900,4.4,1,0,...,0,1,1,0,0,0,1,0,0,0
997,31.84,1,1.5920,33.4320,31.84,4.761905,1.5920,7.7,0,0,...,0,0,0,0,1,0,0,0,0,1
998,65.82,1,3.2910,69.1110,65.82,4.761905,3.2910,4.1,0,0,...,0,0,1,0,0,0,1,0,0,1


In [59]:
df_train, df_test = train_test_split(dataset, test_size = 0.20, shuffle = False)

In [60]:
print (len(df_test))

200


In [61]:
print (len(df_train))

800


#### Arquitectura de la Red

In [62]:
X = df_train.iloc[:, df_train.columns != 'Quantity']
y = df_train.loc[:, 'Quantity']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True)

In [63]:
X_train.shape[1]
inputs = X_train.shape[1]

## Arquitectura 1
Optimización 'SGD'
Redes 6

In [64]:
#base secuencial para definir la estructura de la Red (SIEMPRE INICIAR CON SQUENTIAL)
clasificador = Sequential()

#primera capa oclta
clasificador.add(Dense(input_dim = inputs, units=6, activation='relu', kernel_initializer='uniform'))

#segunda capa oculta
clasificador.add(Dense(units=5, activation='relu', kernel_initializer='uniform'))

#capa de salida
clasificador.add(Dense(units=1, activation='sigmoid', kernel_initializer='uniform'))

#parametros de optimización
clasificador.compile(optimizer='SGD', loss='binary_crossentropy', metrics=['accuracy'])


In [65]:
#parametros de entrenamiento.
clasificador.fit(X_train, y_train, batch_size=25, epochs=150)

Epoch 1/150
26/26 [==============================] - 1s 1ms/step - loss: nan - accuracy: 0.0172
Epoch 2/150
26/26 [==============================] - 0s 2ms/step - loss: nan - accuracy: 0.0000e+00
Epoch 3/150
26/26 [==============================] - 0s 1ms/step - loss: nan - accuracy: 0.0000e+00
Epoch 4/150
26/26 [==============================] - 0s 1ms/step - loss: nan - accuracy: 0.0000e+00
Epoch 5/150
26/26 [==============================] - 0s 2ms/step - loss: nan - accuracy: 0.0000e+00
Epoch 6/150
26/26 [==============================] - 0s 2ms/step - loss: nan - accuracy: 0.0000e+00
Epoch 7/150
26/26 [==============================] - 0s 1ms/step - loss: nan - accuracy: 0.0000e+00
Epoch 8/150
26/26 [==============================] - 0s 1ms/step - loss: nan - accuracy: 0.0000e+00
Epoch 9/150
26/26 [==============================] - 0s 2ms/step - loss: nan - accuracy: 0.0000e+00
Epoch 10/150
26/26 [==============================] - 0s 1ms/step - loss: nan - accuracy: 0.0000e+00
Epo

In [66]:
# parametros de Entrenamiento
#clasificador = KerasClassifier(build_fn=red_v1, batch_size=25, nb_epoch=500)


### Arquitectura 2
Optimización "ADAM"
Cantidad de redes = 6

In [67]:

#base secuencial para definir la estructura de la Red (SIEMPRE INICIAR CON SQUENTIAL)
clasificador2 = Sequential()

#primera capa oclta
clasificador2.add(Dense(input_dim = inputs, units=6, activation='relu', kernel_initializer='uniform'))

#segunda capa oculta
clasificador2.add(Dense(units=5, activation='relu', kernel_initializer='uniform'))

#capa de salida
clasificador2.add(Dense(units=1, activation='sigmoid', kernel_initializer='uniform'))

#parametros de optimización
clasificador2.compile(optimizer='Adam', loss='binary_crossentropy', metrics=['accuracy'])

In [68]:
#parametros de entrenamiento.
clasificador2.fit(X_train, y_train, batch_size=25, epochs=150)

Epoch 1/150
26/26 [==============================] - 1s 2ms/step - loss: 0.3508 - accuracy: 0.0984 
Epoch 2/150
26/26 [==============================] - 0s 2ms/step - loss: -2.5631 - accuracy: 0.1078
Epoch 3/150
26/26 [==============================] - 0s 2ms/step - loss: -14.8727 - accuracy: 0.1078
Epoch 4/150
26/26 [==============================] - 0s 2ms/step - loss: -47.7805 - accuracy: 0.1078
Epoch 5/150
26/26 [==============================] - 0s 2ms/step - loss: -114.4142 - accuracy: 0.1078
Epoch 6/150
26/26 [==============================] - 0s 2ms/step - loss: -233.6156 - accuracy: 0.1078
Epoch 7/150
26/26 [==============================] - 0s 2ms/step - loss: -416.9669 - accuracy: 0.1078
Epoch 8/150
26/26 [==============================] - 0s 2ms/step - loss: -690.2485 - accuracy: 0.1078
Epoch 9/150
26/26 [==============================] - 0s 2ms/step - loss: -1063.2914 - accuracy: 0.1078
Epoch 10/150
26/26 [==============================] - 0s 2ms/step - loss: -1543.8167 - 

Arquitectura 3
Optimización "RMSprop" 
Cantidad de redes = 6

In [69]:
#base secuencial para definir la estructura de la Red (SIEMPRE INICIAR CON SQUENTIAL)
clasificador3 = Sequential()

#primera capa oclta
clasificador3.add(Dense(input_dim = inputs, units=6, activation='relu', kernel_initializer='uniform'))

#segunda capa oculta
clasificador3.add(Dense(units=6, activation='relu', kernel_initializer='uniform'))

#capa de salida
clasificador3.add(Dense(units=1, activation='sigmoid', kernel_initializer='uniform'))

#parametros de optimización
clasificador3.compile(optimizer='RMSprop', loss='binary_crossentropy', metrics=['accuracy'])

In [70]:
#parametros de entrenamiento.
clasificador3.fit(X_train, y_train, batch_size=100, epochs=500)

Epoch 1/500
7/7 [==============================] - 1s 3ms/step - loss: 0.5430 - accuracy: 0.1078
Epoch 2/500
7/7 [==============================] - 0s 2ms/step - loss: 0.0334 - accuracy: 0.1078
Epoch 3/500
7/7 [==============================] - 0s 2ms/step - loss: -0.7405 - accuracy: 0.1078
Epoch 4/500
7/7 [==============================] - 0s 2ms/step - loss: -1.8426 - accuracy: 0.1078
Epoch 5/500
7/7 [==============================] - 0s 2ms/step - loss: -3.3505 - accuracy: 0.1078
Epoch 6/500
7/7 [==============================] - 0s 2ms/step - loss: -5.2262 - accuracy: 0.1078
Epoch 7/500
7/7 [==============================] - 0s 2ms/step - loss: -7.5077 - accuracy: 0.1078
Epoch 8/500
7/7 [==============================] - 0s 2ms/step - loss: -10.2470 - accuracy: 0.1078
Epoch 9/500
7/7 [==============================] - 0s 3ms/step - loss: -13.4426 - accuracy: 0.1078
Epoch 10/500
7/7 [==============================] - 0s 2ms/step - loss: -16.9813 - accuracy: 0.1078
Epoch 11/500
7/7 [

## Ariquitectura 4
Optimización: 'Adagrad'
redes 6


```
# This is formatted as code
```



In [71]:
#base secuencial para definir la estructura de la Red (SIEMPRE INICIAR CON SQUENTIAL)
clasificador4 = Sequential()

#primera capa oclta
clasificador4.add(Dense(input_dim = inputs, units=6, activation='relu', kernel_initializer='uniform'))

#segunda capa oculta
clasificador4.add(Dense(units=6, activation='relu', kernel_initializer='uniform'))

#capa de salida
clasificador4.add(Dense(units=1, activation='sigmoid', kernel_initializer='uniform'))

#parametros de optimización
clasificador4.compile(optimizer='Adagrad', loss='binary_crossentropy', metrics=['accuracy'])

In [72]:
#parametros de entrenamiento.
clasificador4.fit(X_train, y_train, batch_size=25, epochs=150)

Epoch 1/150
26/26 [==============================] - 1s 2ms/step - loss: 0.5024 - accuracy: 0.1063 
Epoch 2/150
26/26 [==============================] - 0s 2ms/step - loss: 0.0983 - accuracy: 0.1078
Epoch 3/150
26/26 [==============================] - 0s 2ms/step - loss: -0.2336 - accuracy: 0.1078
Epoch 4/150
26/26 [==============================] - 0s 1ms/step - loss: -0.5669 - accuracy: 0.1078
Epoch 5/150
26/26 [==============================] - 0s 2ms/step - loss: -0.9137 - accuracy: 0.1078
Epoch 6/150
26/26 [==============================] - 0s 2ms/step - loss: -1.3061 - accuracy: 0.1078
Epoch 7/150
26/26 [==============================] - 0s 2ms/step - loss: -1.8286 - accuracy: 0.1078
Epoch 8/150
26/26 [==============================] - 0s 2ms/step - loss: -2.4329 - accuracy: 0.1078
Epoch 9/150
26/26 [==============================] - 0s 2ms/step - loss: -3.1114 - accuracy: 0.1078
Epoch 10/150
26/26 [==============================] - 0s 2ms/step - loss: -3.8577 - accuracy: 0.1078


## conclusion:
Los métodos que generaron un accuracy  similar son: Adam, Adagrad, RMSprop.

In [73]:
# Guardardamos la red.
path = '/content/drive/MyDrive/Statistical/Laboratorio2/' 
clasificador3.save(path + '6_6_RMSprop_out.h5')

In [74]:
# Cargamos la red.
new_model = keras.models.load_model(path + '6_6_RMSprop_out.h5')

In [75]:
#predicciones
y_preds = new_model.predict(X_test)
y_preds

array([[1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],

In [76]:
y_preds = (y_preds >= 0.5)
y_preds

array([[ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [

In [77]:
# Métricas
accuracy_score(y_test, y_preds)

0.10625

## Kfolds

In [78]:
def red_v1():
  #base secuencial para definir la estructura de la Red (SIEMPRE INICIAR CON SQUENTIAL)
  clasificador5 = Sequential()

  #primera capa oclta
  clasificador5.add(Dense(input_dim = 6, units=6, activation='relu', kernel_initializer='uniform'))

  #segunda capa oculta
  clasificador5.add(Dense(units=6, activation='relu', kernel_initializer='uniform'))

  #capa de salida
  clasificador5.add(Dense(units=1, activation='sigmoid', kernel_initializer='uniform'))

  #parametros de optimización
  clasificador5.compile(optimizer='SGD', loss='binary_crossentropy', metrics=['accuracy'])

  return clasificador5

In [79]:
clasificador5 = KerasClassifier(build_fn=red_v1, batch_size=25, nb_epoch=150)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  """Entry point for launching an IPython kernel.


In [90]:
kfolds = KFold(n_splits=10, shuffle=True)
accs = cross_val_score(clasificador5, X=X_train, y=y_train, cv=kfolds, verbose=1, n_jobs=-1)

# Gred search 

In [81]:
from sklearn.model_selection import GridSearchCV
import time

In [82]:
parametros = {
    'batch_size': [5, 100],
    'nb_epoch':[100, 200],
    'optimizer': ['adam', 'RMSprop']
}

In [83]:
def red_grid_search_v1(optimizer):
  clasificador = Sequential()
  clasificador.add(Dense(input_dim = 11, units=6, activation='relu', kernel_initializer='uniform'))
  clasificador.add(Dense(units=6, activation='relu', kernel_initializer='uniform'))
  clasificador.add(Dense(units=1, activation='sigmoid', kernel_initializer='uniform'))
  clasificador.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
  return clasificador

In [84]:
clasificador_grid_search = KerasClassifier(build_fn=red_grid_search_v1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  """Entry point for launching an IPython kernel.


In [85]:
grid_out = GridSearchCV(estimator=clasificador_grid_search,
                        param_grid=parametros,
                        cv=20,
                        scoring='accuracy')

In [90]:
start = time.time()
train_gs_out = grid_out.fit(X_train, y_train, verbose=1)
end = time.time()
total = end - start
print("total entranamiento: ", total)

In [87]:
grid_out.best_params_

{'batch_size': 5, 'nb_epoch': 100, 'optimizer': 'adam'}

In [88]:
grid_out.best_score_

nan

## ANN para Regresiones

In [93]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [95]:
X = dataset.iloc[:, dataset.columns != 'Quantity']
y = dataset.loc[:, 'Quantity']

In [96]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle=True, random_state=800)

In [97]:
std_scaler = StandardScaler()

X_train = std_scaler.fit_transform(X_train)
X_test = std_scaler.transform(X_test)

X_train.shape, X_test.shape

((700, 20), (300, 20))

In [99]:
from tensorflow.keras.optimizers import Adam

In [101]:
entradas = X_train.shape[1]
lr = 0.05
batch_sz=100

regressor = Sequential()
regressor.add(Dense(input_dim=entradas, units=6, kernel_initializer='uniform', activation='relu'))
regressor.add(Dense(units=6, kernel_initializer='uniform', activation='relu'))
regressor.add(Dense(units=7, kernel_initializer='uniform', activation='relu'))
regressor.add(Dense(units=1, kernel_initializer='uniform', activation='relu'))
regressor.compile(optimizer=Adam(lr), loss='mean_squared_error', metrics=['mean_squared_error']) #regresion.
result = regressor.fit(X_train, y_train, batch_size=batch_sz, validation_split=0.2, verbose=1, epochs=100)

Epoch 1/100
6/6 [==============================] - 1s 66ms/step - loss: 35.2246 - mean_squared_error: 35.2246 - val_loss: 18.5079 - val_mean_squared_error: 18.5079
Epoch 2/100
6/6 [==============================] - 0s 14ms/step - loss: 14.5924 - mean_squared_error: 14.5924 - val_loss: 5.9326 - val_mean_squared_error: 5.9326
Epoch 3/100
6/6 [==============================] - 0s 13ms/step - loss: 7.7387 - mean_squared_error: 7.7387 - val_loss: 4.3287 - val_mean_squared_error: 4.3287
Epoch 4/100
6/6 [==============================] - 0s 13ms/step - loss: 6.0958 - mean_squared_error: 6.0958 - val_loss: 4.2444 - val_mean_squared_error: 4.2444
Epoch 5/100
6/6 [==============================] - 0s 16ms/step - loss: 4.2634 - mean_squared_error: 4.2634 - val_loss: 3.0554 - val_mean_squared_error: 3.0554
Epoch 6/100
6/6 [==============================] - 0s 14ms/step - loss: 3.7028 - mean_squared_error: 3.7028 - val_loss: 2.8649 - val_mean_squared_error: 2.8649
Epoch 7/100
6/6 [=================

In [102]:
result.history

{'loss': [35.2246208190918,
  14.592382431030273,
  7.738651752471924,
  6.09578275680542,
  4.2634172439575195,
  3.702763080596924,
  2.657517910003662,
  2.2195746898651123,
  1.8237985372543335,
  1.6702336072921753,
  1.595383882522583,
  1.5029557943344116,
  1.4801362752914429,
  1.4665886163711548,
  1.35114586353302,
  1.3206547498703003,
  1.2938731908798218,
  1.2373740673065186,
  1.2214637994766235,
  1.1666772365570068,
  1.1188827753067017,
  1.1081863641738892,
  1.1570136547088623,
  1.2292732000350952,
  1.1046119928359985,
  1.0009123086929321,
  0.9635905623435974,
  0.9487999081611633,
  0.8979985117912292,
  1.0485235452651978,
  0.9667121767997742,
  0.8322793245315552,
  0.8455623984336853,
  0.8201635479927063,
  0.8505469560623169,
  0.8978931307792664,
  0.9214187860488892,
  0.9350313544273376,
  0.9639902114868164,
  0.9333855509757996,
  0.8488792181015015,
  0.8490723371505737,
  0.865653932094574,
  0.8510187268257141,
  0.8630810379981995,
  0.844631373

In [103]:
from keras.layers import Activation
from keras.wrappers.scikit_learn import KerasClassifier
from keras.wrappers.scikit_learn import KerasRegressor

In [104]:
## Generar Arquitectura Dinámica
def create_dynamicNeuralNet(layers, activation_fn, tipo, lr, n_inputs, n_clases):
  modelo = Sequential()

  for i, nodes in enumerate(layers):
    if(i == 0):
      modelo.add(Dense(units=nodes, input_dim=n_inputs[0]))
      modelo.add(Activation(activation_fn))
    else:
      modelo.add(Dense(units=nodes))
      modelo.add(Activation(activation_fn))

    if(tipo == "regresion"):
      modelo.compile(loss="mean_squared_error", optimizer=Adam(lr), metrics=['mean_squared_error'])
      out_activation='relu'
      classes_out = 1
    else:
      modelo.compile(loss="binary_crossentropy", optimizer=Adam(lr), metrics=['accuracy'])
      out_activation='sigmoid'
      classes_out = n_clases[0]
      if(classes_out == 2):
        classes_out = 1
    modelo.add(Dense(units=classes_out, activation=out_activation))
    return modelo


In [105]:
parametros = {
    'batch_size': [50],
    'activation_fn': ['relu'],
    'epochs': [100],
    'lr': [0.1],
    'layers': [(20), (40, 20), (10, 15, 10)],
    'tipo': ['regresion'],
    'n_inputs': [X_train.shape[1]],
    'n_clases': [len(y_train.value_counts())]
}

In [106]:
for i, val in enumerate([(20), (40, 20), (10, 15, 10)]):
  print(val)

20
(40, 20)
(10, 15, 10)


In [107]:
parametros

{'activation_fn': ['relu'],
 'batch_size': [50],
 'epochs': [100],
 'layers': [20, (40, 20), (10, 15, 10)],
 'lr': [0.1],
 'n_clases': [10],
 'n_inputs': [20],
 'tipo': ['regresion']}

In [111]:
grid_out = GridSearchCV(estimator=modelofinal, param_grid=parametros)

In [110]:
grid_result = grid_out.fit(X_train, y_train, random_state=2022)